<a href="https://colab.research.google.com/github/Omotola203/-Bettercap-HQ-Pen-Testing/blob/main/HERRY'S_PHISHSENTINEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
import re
import os
from urllib.parse import urlparse
import joblib
import pandas as pd
from requests import get, RequestException
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Define the list of phishing indicators
phishing_indicators = [
    "login.php", "member.php", "register.php", "forgot.php", "change.php",
    "account.php", "password.php", "profile.php", "update.php",
    "password-recovery.php", "recover.php", "reset.php", "retrieve.php"
]

# Load known safe domains from the 'top-1m.csv' file
def load_safe_domains():
    try:
        df = pd.read_csv('top-1m.csv', header=None, encoding='ISO-8859-1')
        safe_domains = df[1].tolist()  # The domains are in the second column
    except FileNotFoundError:
        print("Error: 'top-1m.csv' file not found.")
        safe_domains = [
   "google.com", "wikipedia.org", "amazon.com", "github.com", "microsoft.com", "youtube.com", "bbc.com", "nytimes.com",
"coursera.org", "stackoverflow.com", "deepai.org", "spotify.com", "slack.com", "zoom.us", "cloudflare.com", "godaddy.com",
"bbc.com", "cnn.com", "nytimes.com", "theguardian.com", "forbes.com", "bloomberg.com", "wsj.com", "nationalgeographic.com",
"weather.com", "airbnb.com", "uber.com", "lyft.com", "booking.com", "expedia.com", "tripadvisor.com", "nike.com",
"adidas.com", "zara.com", "hm.com", "sephora.com", "macys.com", "walmart.com", "target.com", "costco.com",
"bestbuy.com", "homedepot.com", "lowes.com", "ikea.com", "wholefoodsmarket.com", "walgreens.com", "cvs.com", "samsung.com",
"sony.com", "lg.com", "dell.com", "hp.com", "asus.com", "lenovo.com", "acer.com", "msi.com",
"razer.com", "logitech.com", "autodesk.com", "sketchup.com", "blender.org", "zendesk.com", "freshdesk.com", "trello.com",
"asana.com", "jira.com", "bitbucket.org", "toggl.com", "hubspot.com", "mailchimp.com", "constantcontact.com", "activecampaign.com",
"getresponse.com", "aweber.com", "sendinblue.com", "zoho.com", "monday.com", "smartsheet.com", "clickup.com", "wrike.com",
"basecamp.com", "notion.so", "evernote.com", "onenote.com", "todoist.com", "ticktick.com", "habitica.com", "toggl.com",
"rescuetime.com", "myhours.com", "clockify.me", "timecamp.com", "harvestapp.com", "freckle.com", "hourstack.com", "timedoctor.com",
"hubstaff.com", "paymoapp.com", "tickspot.com", "payscale.com", "glassdoor.com", "indeed.com", "monster.com", "ziprecruiter.com",
"angel.co", "linkedin.com/jobs", "simplyhired.com", "careerbuilder.com", "dice.com", "hired.com", "thebalancecareers.com", "roberthalf.com",
"randstadusa.com", "adeccousa.com", "kellyservices.us", "manpower.com", "expresspros.com", "careerarc.com", "snagajob.com", "wayup.com",
"collegegrad.com", "internships.com", "govtjobs.com", "federaljobs.net", "usajobs.gov", "state.gov", "cityjobs.com", "countyjobs.com",
"districtjobs.com", "publicservicecareers.org", "teachercatapult.com", "educationamerica.net", "higheredjobs.com", "chroniclevitae.com", "k12jobspot.com", "academiccareers.com",
"employmentcrossing.com", "nationjob.com", "jobbankusa.com", "jobhero.com", "jobfox.com", "job-hunt.org", "localwise.com", "meetup.com",
"eventbrite.com", "tickettailor.com", "peerspace.com", "breather.com", "wework.com", "regus.com", "knotel.com", "convene.com",
"industriousoffice.com", "davincivirtual.com", "opusoffices.com", "cloudvo.com", "liquidspace.com", "croissant.io", "workfrom.co", "workbar.com",
"serendipitylabs.com", "workthere.com", "servcorp.com", "spacesworks.com", "coworker.com", "coworkingresources.org", "workspot.com", "maple.com",
"robinpowered.com", "eden.io", "officeotp.com", "birdoffice.com", "optixapp.com", "deskpass.com", "myhq.in", "hubblehq.com",
"baremetrics.com", "chartmogul.com", "profitwell.com", "clientbooks.com", "pulseapp.com", "bill.com", "xero.com", "quickbooks.intuit.com",
"freshbooks.com", "waveapps.com", "sage.com", "kashoo.com", "outright.com", "freeagent.com", "zipbooks.com", "zoho.com/books",
"lessaccounting.com", "invoiceninja.com", "and.co", "bokio.co.uk", "invoicehome.com", "invoiceberry.com", "invoice2go.com", "simpleinvoices.com",
"sliqtools.co.uk", "free-invoice.co.uk", "streetinvoice.com", "invoicely.com", "cashboardapp.com", "getharvest.com", "timecamp.com", "toggl.com",
"myhours.com", "clockify.me", "hubstaff.com", "worksnaps.net", "workpuls.com", "desktime.com", "paymoapp.com", "timewatch.com",
"tsheets.com", "timedoctor.com", "rescuetime.com", "timeular.com", "teamwork.com", "asana.com", "monday.com", "wrike.com",
"clickup.com", "notion.so", "trello.com", "basecamp.com", "smartsheet.com", "airtable.com", "qube-os.com", "flow.com",
"getflow.com", "teamgantt.com", "goodday.work", "proofhub.com", "taskworld.com", "azendoo.com", "projectplace.com", "mavenlink.com",
"nutcache.com", "redbooth.com", "scoro.com", "kanbanflow.com", "kanbanchi.com", "taskque.com", "workzone.com", "clarizen.com",
"liquidplanner.com", "workfront.com", "targetprocess.com", "daptiv.com", "projectinsight.net", "functionfox.com", "projectmanager.com", "project-open.com",
"ganttpro.com", "easyprojects.net", "sciforma.com", "bigtime.net", "teamworkpm.net", "celoxis.com", "appfluence.com", "prioritymatrix.com",
"viewpath.com", "rindle.com", "hey.space", "volerro.com", "activecollab.com", "taskade.com", "glasscubes.com", "plan.io",
"clockodo.com", "intervals.com", "breeze.pm", "planzone.com", "bubbl.us", "mindmeister.com", "coggle.it", "xmind.net",
"mindmup.com", "thebrain.com", "stormboard.com", "popplet.com", "mindomo.com", "simplemind.eu", "mindlyapp.com", "freemind.sourceforge.net",
"mindmapper.com", "mapul.com", "mindgenius.com", "inspiration.com", "conceptdraw.com", "creately.com", "smartdraw.com", "lucidchart.com",
"gliffy.com", "draw.io", "diagrams.net", "cacoo.com", "whimsical.com", "canva.com", "visme.co", "piktochart.com",
"venngage.com", "infogram.com", "easel.ly", "visualize.me", "adobe.com/products/illustrator", "coreldraw.com", "sketch.com", "affinity.serif.com",
"inkscape.org", "gravit.io", "vectornator.io", "vectr.com", "svg-edit.googlecode.com", "autodesk.com/products/autocad", "sketchup.com", "tinkercad.com",
"3ds.com", "blender.org", "freecadweb.org", "opencascade.com", "onshape.com", "fusion360.autodesk.com", "solidworks.com", "ironcad.com",
"rhino3d.com", "solidthinking.com", "siemens.com/software/solid-edge", "vectorworks.net", "bentley.com", "3d-coat.com", "mari.org", "keyshot.com",
"lumion.com", "unrealengine.com", "unity3d.com", "cryengine.com", "godotengine.org", "houdini.com", "marmoset.co", "substance3d.com",
"adobe.com/products/dimension", "zbrushcentral.com", "autodesk.com/products/maya", "pixologic.com", "maxon.net", "planetside.co.uk", "terragen.com", "world-machine.com",
"gaea.mydaxdev.com", "vue.bentley.com", "quixel.com", "megascans.se", "bridge.se", "mixer.se", "allegorithmic.com", "substance3d.com",
"x-normal.net", "knaldtech.com", "crazybump.com", "microsoft.com/pt-br/windows/photos", "adobe.com/products/photoshop", "gimp.org", "krita.org", "paint

        ]
    except UnicodeDecodeError as e:
        print(f"Error: {e}. Could not decode 'top-1m.csv'.")
        safe_domains = []
    return safe_domains

# List of known safe domains
safe_domains = load_safe_domains()

def extract_features(url):
    features = {}
    parsed_url = urlparse(url)
    domain = parsed_url.netloc

    # Feature: Domain length
    features['domain_length'] = len(domain)

    # Feature: URL length
    features['url_length'] = len(url)

    # Feature: Count of dots in domain
    features['dot_count'] = domain.count('.')

    # Feature: Count of phishing indicators in path
    features['phishing_indicators_count'] = sum(indicator in parsed_url.path for indicator in phishing_indicators)

    return features

def train_model(dataset_path):
    # Load the cleaned dataset
    try:
        df = pd.read_csv(dataset_path, low_memory=False, encoding='ISO-8859-1')  # or try 'cp1252'
    except FileNotFoundError:
        print(f"Error: Dataset file not found at '{dataset_path}'. Please check the file path and try again.")
        return
    except pd.errors.ParserError as e:
        print(f"Error: Dataset could not be parsed. Please check the file format and try again. ParserError: {e}")
        return
    except UnicodeDecodeError as e:
        print(f"Error: {e}. Could not decode the dataset file.")
        return

    # Ensure the dataset contains 'url' and 'label' columns
    if 'url' not in df.columns or 'label' not in df.columns:
        print("Error: The dataset must contain 'url' and 'label' columns.")
        return

    # Extract features for each URL
    feature_df = pd.DataFrame(df['url'].apply(extract_features).tolist())

    # Create feature matrix and label vector
    X = pd.concat([df, feature_df], axis=1).drop(columns=['url', 'label'])
    y = df['label']

    # Split the data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train a random forest classifier
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    # Save the trained model
    joblib.dump(model, 'phishing_classifier.pkl')

def detect_phishing(url):
    parsed_url = urlparse(url)
    domain = parsed_url.netloc

    # Check if the domain is in the list of known safe domains
    if any(domain.endswith(safe_domain) for safe_domain in safe_domains):
        return False

    # Check if the URL contains any phishing indicators in its path
    if any(indicator in parsed_url.path for indicator in phishing_indicators):
        return True

    try:
        # Send a request to the URL and get the HTML response
        response = get(url, timeout=10)
        response.raise_for_status()
    except RequestException:
        return True  # Treat any request issues as suspicious

    soup = bs4.BeautifulSoup(response.content, 'html.parser')

    # Check if the page has suspicious HTML content
    content = soup.get_text()
    if re.search(r"password|username|email|account|login|logout|register|forgot", content, re.IGNORECASE):
        return True

    # Check if the URL has a suspicious file extension
    file_extension = os.path.splitext(parsed_url.path)[1]
    if file_extension in [".exe", ".zip", ".rar", ".tar", ".gz"]:
        return True

    # Extract features for the URL
    features = extract_features(url)
    features_df = pd.DataFrame([features])

    # Load the trained model
    model = joblib.load('phishing_classifier.pkl')

    # Predict using the trained model
    is_phishing = model.predict(features_df)[0]

    return bool(is_phishing)

def main():
    # Path to your cleaned dataset file
    cleaned_dataset_path = 'cleaned_dataset_phishing.csv'

    # Train the model with the specified dataset
    train_model(dataset_path=cleaned_dataset_path)

    print("Welcome to Herry's PhishSentinel, your vigilant guardian against phishing threats. Protecting your digital world, one click at a time.")
    while True:
        # Get the URL from the user
        url = input("Enter the URL to check (or type 'exit' to quit): ").strip()

        # Check if the user wants to exit
        if url.lower() == 'exit':
            print("Exiting the phishing detector. Looking forward to seeing you again. Stay safe and goodbye!")
            break

        # Detect phishing attack
        is_phishing = detect_phishing(url)

        # Print the result
        if is_phishing:
            print(f"{url} is a phishing URL!")
        else:
            print(f"{url} is not a phishing URL!")

if __name__ == "__main__":
    main()


Welcome to Herry's PhishSentinel, your vigilant guardian against phishing threats. Protecting your digital world, one click at a time.
Enter the URL to check (or type 'exit' to quit): https://deepai.org
https://deepai.org is a phishing attack!
Enter the URL to check (or type 'exit' to quit): https://www.kaggle.com
https://www.kaggle.com is a phishing attack!
Enter the URL to check (or type 'exit' to quit): https://www.razer.com
https://www.razer.com is not a phishing attack.
Enter the URL to check (or type 'exit' to quit): exit
Exiting the phishing detector. Looking forward to seeing you again. Stay safe and goodbye!
